# Setup 

In [ ]:
import google.generativeai as gai
import json
# Gemini API Key import
gemini_key = json.load(open('keys.json'))['gemini']['api-key']
gai.configure(api_key=gemini_key)
MODEL = gai.GenerativeModel('gemini-pro')
import sqlite3 as sql

# Define helper functions

In [ ]:
def is_sql(line:str):
    first_word = line.strip().split(" ")[0]
    return first_word == first_word.upper()

In [ ]:
def reformat_response(resp):
    return "\n".join([line for line in resp.text.split('\n') if line.split()[0]==line.split()[0].upper() and line!="\n" and "`" not in line])

In [ ]:
def get_sql_from_llm(assignment,model):
    chat = model.start_chat(history=[])
    return chat.send_message(f'You have a database with the following tables: {tables}. Those tables have the following columns: {table_columns}. Generate a sql query with each command on a different line such that it satisfies the following: '+assignment+ ". Do not explain and give no other text. If such a query does not make sense in the context of the database or if any requested columns do not exist return the phrase 'Invalid prompt'."), chat

In [ ]:
def execute_query(query:str, cur):
    try:
        return cur.execute(query).fetchall()
    except:
        return "Invalid prompt"

In [ ]:
def get_instructions(assignment, model, cur):
    response, chat = get_sql_from_llm(assignment,model)
    if "Invalid prompt" in response.text or not any([is_sql(line) for line in response.text.split('\n')]):
        return ["None"],["Please enter a question that makes sense in the context of the database."], chat
    instructions = []
    query = "\n".join([line for line in reformat_response(response).split('\n') if is_sql(line)])
    invalid, try_count = True,0
    while invalid and try_count<5:
        results = execute_query(query, cur)
        if results == "Invalid prompt":
            print(f"The generated query: {query} did not make sense in the context of the database. Trying new query.")
            response, chat = get_sql_from_llm(assignment,model)
            query = "\n".join([line for line in reformat_response(response).split('\n') if is_sql(line)])
        else:
            invalid = False
    if invalid:
        return ["None"],["A valid query could not be generated. Please try again."], chat
    if len(results)<1:
        return ["None"],["The generated query did not return any results. Please enter a valid prompt."], chat
    for query_line in query.split('\n'):
        if is_sql(query_line):
            instructions+= [query_line+" ["+chat.send_message("Explain what the command "+query_line+"is doing in one sentence on one line without any formatting. Be specific about any tables, columns, or values that are used.").text.split("\n")[0]+"]"]
    return results, instructions, chat

In [ ]:
def show_instructions_and_results(instructions, results):
    ret = "Results: \n"
    for r, result in enumerate(results):
        ret+=f"{result.__str__()}\n"
    ret += f"Instructions: \n"
    for i, instruction in enumerate(instructions):
        ret+=f"{instruction}\n"
    return ret

# Run

In [ ]:
import requests
import sqlite3, os
# URL of the SQLite database file
url = input("Enter the URL of the SQLite database file: ").replace("blob", "raw")

# Download the database file
response = requests.get(url).content if 'http' in url else open(url, "rb").read()
with open("temp.db", "wb") as file:
    file.write(response)

# Connect to the downloaded database file
conn = sqlite3.connect("temp.db")
cur = conn.cursor()
# Create a list of all the tables in the database
tables = [table_tuple[0] for table_tuple in cur.execute("SELECT name FROM sqlite_master  WHERE type='table';").fetchall()]
# Create a dictionary of all the columns in each table in the database
table_columns = {}
for table in tables:
    table_columns[table] = [column[0] for column in cur.execute(f"SELECT * FROM {table}").description]
prompt = input("Enter the prompt: ")
results,instructions,chat = get_instructions(prompt, MODEL,cur)
print(show_instructions_and_results(instructions, results))